# Load IHOP Data
The data is stored with a location on each line as a JSON blob. Format the location to make calculations easier.

In [1]:
import json

ALL_IHOPS = [json.loads(line) for line in open('ihop_list.jl')]
for ihop in ALL_IHOPS:
    ihop.update({'location': (float(ihop['latitude']), float(ihop['longitude']))})

IHOP_DICT = dict((x['store_id'], x) for x in ALL_IHOPS)

# Display IHOP Locations

In [2]:
from ipyleaflet import Map, CircleMarker
from IPython.core.display import display

ihop_map = Map(center=(37.8, -96), zoom=4)

for ihop in ALL_IHOPS:
    ihop_map.add_layer(CircleMarker(
        location=ihop['location'],
        title=ihop['address'],
        draggable=False,
        radius=1,
    ))

display(ihop_map)

Map(center=[37.8, -96], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

# Calculate IHOP Densities
For any given IHOP, how far away is the Xth closest IHOP?

In [3]:
from geopy.distance import geodesic
import multiprocess

NUM_LOCATIONS = 25


def closest_ihops(start_ihop, ihops):
    return sorted(ihops, key=lambda other_ihop: geodesic(start_ihop['location'], other_ihop['location']))


def ihop_radius(start_ihop):
    sorted_ihops = closest_ihops(start_ihop, ALL_IHOPS)
    return geodesic(start_ihop['location'], sorted_ihops[NUM_LOCATIONS]['location']).miles


def rank_ihops():
    pool = multiprocess.Pool()
    radiuses = pool.map_async(ihop_radius, ALL_IHOPS).get()
    ihops_with_distances = list(zip(ALL_IHOPS, radiuses))
    ranked_ihops_with_distances = sorted(ihops_with_distances, key=lambda x: x[1])
    return list(map(lambda x: x[0], ranked_ihops_with_distances))


RANKED_IHOPS = rank_ihops()

# Visualize IHOP Densities

In [4]:
from ipyleaflet import Map, CircleMarker
from IPython.core.display import Markdown, display
from geopy.distance import geodesic
import math


def highlight_ihop(featured_ihop):
    neighbor_ihops = closest_ihops(featured_ihop, ALL_IHOPS)[1:NUM_LOCATIONS]

    ihop_map = Map(center=featured_ihop['location'], zoom=10)

    ihop_map.add_layer(CircleMarker(
        location=featured_ihop['location'],
        title=featured_ihop['address'],
        draggable=False,
        radius=1,
        color='red',
    ))

    for ihop in neighbor_ihops:
        ihop_map.add_layer(CircleMarker(
            location=ihop['location'],
            title=ihop['address'],
            draggable=False,
            radius=1,
        ))

    display(Markdown('# ' + featured_ihop['address']))
    radius = geodesic(featured_ihop['location'], neighbor_ihops[-1]['location']).miles
    display(Markdown('radius: %s miles' % (radius)))
    display(ihop_map)

In [5]:
def top_markets(num_markets):
    RANKED_IDS = list(map(lambda x: x['store_id'], RANKED_IHOPS))
    # Remove the DISQUALIFY_COUNT every time cycling through
    DISQUALIFY_COUNT = 100
    visited_ihop_ids = set([])
    index = 0
    for _ in range(num_markets):
        if index >= len(RANKED_IHOPS):
            return
        else:
            top_ihop_id = RANKED_IDS[index]
            yield IHOP_DICT[top_ihop_id]

            closest = map(lambda x: x['store_id'], closest_ihops(IHOP_DICT[top_ihop_id], ALL_IHOPS)[:DISQUALIFY_COUNT])
            visited_ihop_ids = set(closest).union(visited_ihop_ids)

            while index < len(RANKED_IHOPS) and RANKED_IDS[index] in visited_ihop_ids:
                index = index + 1


for ihop in top_markets(5):
    highlight_ihop(ihop)


# 235 E 14th St # 237, New York, NY, 10003

radius: 11.145798559236592 miles

Map(center=[40.73278, -73.98557], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

# 6300 Santa Fe Ave, Huntington Park, CA, 90255

radius: 11.148020070668327 miles

...

radius: 16.01241027533158 miles

Map(center=[38.84607, -76.98265], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

# 6508 Washington Ave, Houston, TX, 77007

radius: 18.1462937283423 miles

Map(center=[29.77658, -95.4273], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [6]:
highlight_ihop(RANKED_IHOPS[-1])

# 3269 Kaopaka St, Honolulu, HI, 96819

radius: 2413.849239875795 miles

Map(center=[21.335151, -157.914522], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…